## Scarping insolvency data
Source: https://www.insolvenzbekanntmachungen.de/cgi-bin/bl_suche.pl

1. 'Uneingeschränkte Suche': search for all insolvency court but only for 2 weeks after publication date
2. 'Detail-Suche': 
    * input: insolvency court + company name OR residence of the debtor



In [2]:
# install required packages:
#!pip install requests
#!pip install bs4
#!pip install pandas
#!pip install progressbar

In [3]:
# import required packages:
import os
import json
import helpers
import time
import pandas as pd

# 1. Get regions and insolvency court from
www.insolvenzbekanntmachungen.de:

In [4]:
insolvency_court = helpers.scrape_insolvency_court()

In [5]:
insolvency_court

{'Baden-Württemberg': ['Aalen',
  'Baden-Baden',
  'Crailsheim',
  'Esslingen',
  'Freiburg',
  'Göppingen',
  'Hechingen',
  'Heidelberg',
  'Heilbronn',
  'Karlsruhe',
  'Konstanz',
  'Ludwigsburg',
  'Lörrach',
  'Mannheim',
  'Mosbach',
  'Offenburg',
  'Pforzheim',
  'Ravensburg',
  'Rottweil',
  'Stuttgart',
  'Tübingen',
  'Ulm',
  'Villingen-Schwenningen',
  'Waldshut-Tiengen'],
 'Bayern': ['Amberg',
  'Ansbach',
  'Aschaffenburg',
  'Augsburg',
  'Bamberg',
  'Bayreuth',
  'Coburg',
  'Deggendorf',
  'Fürth',
  'Hof',
  'Ingolstadt',
  'Kempten',
  'Landshut',
  'Memmingen',
  'Mühldorf',
  'München',
  'Neu-Ulm',
  'Nördlingen',
  'Nürnberg',
  'Passau',
  'Regensburg',
  'Rosenheim',
  'Schweinfurt',
  'Straubing',
  'Traunstein',
  'Weiden',
  'Weilheim',
  'Wolfratshausen',
  'Würzburg'],
 'Berlin': ['Charlottenburg',
  'Köpenick',
  'Lichtenberg',
  'Mitte',
  'Neukölln',
  'Pankow/Weißensee',
  'Schöneberg',
  'Spandau',
  'Tempelhof-Kreuzberg',
  'Tiergarten',
  'Weddin

# 2. Example:

## 2.1 unlimited_search():

In [6]:
# helpers.unlimited_search(region='--+Alle+Bundesl%E4nder+--', court='--+Alle+Insolvenzgerichte+--&', 
#                          matchesperpage=100, verbose=False)

## 2.1 detail_search():

In [6]:
helpers.detail_search(name='Senvion GmbH', region='Hamburg', court='Hamburg', verbose=True)

--- 0.2918977737426758 seconds ---
Number of results: 5


['2020-06-03Senvion GmbH, Hamburg, 67g IN 113/19, Registergericht Hamburg, HRB 137187',
 '2020-04-15Senvion GmbH, Hamburg, 67g IN 113/19, Registergericht Hamburg, HRB 137187',
 '2019-08-22Senvion GmbH, Hamburg, 67g IN 113/19, Registergericht Hamburg, HRB 137187',
 '2019-07-02Senvion GmbH, Hamburg, 67g IN 113/19, Registergericht Hamburg, HRB 137187',
 '2019-07-02Senvion GmbH, Hamburg, 67g IN 113/19, Registergericht Hamburg, HRB 137187']

## 3. Scrape court and insolvency:
www.gerichtsverzeichnis.de

In [7]:
# Test

In [8]:
# test company
test_company = [['Senvion GmbH', 'Senvion GmbH', 'missing', 'missing', '22297', ' Hamburg']]
 
helpers.scrape_court_and_insolvency(test_company, insolvency_court, time_out=0, verbose=True)


0. / 1
Senvion GmbH
22297
Hamburg


Court: Hamburg
Insolvency: Senvion GmbH: 5
0/1


({'Senvion GmbH': ['2020-06-03Senvion GmbH, Hamburg, 67g IN 113/19, Registergericht Hamburg, HRB 137187',
   '2020-04-15Senvion GmbH, Hamburg, 67g IN 113/19, Registergericht Hamburg, HRB 137187',
   '2019-08-22Senvion GmbH, Hamburg, 67g IN 113/19, Registergericht Hamburg, HRB 137187',
   '2019-07-02Senvion GmbH, Hamburg, 67g IN 113/19, Registergericht Hamburg, HRB 137187',
   '2019-07-02Senvion GmbH, Hamburg, 67g IN 113/19, Registergericht Hamburg, HRB 137187']},
 [],
 [['Senvion GmbH',
   'Senvion GmbH',
   'missing',
   'missing',
   '22297',
   ' Hamburg',
   'Hamburg',
   1]])

## 3.1 Load company information

In [7]:
basedir = os.path.normpath(os.getcwd() + os.sep + os.pardir)
basedir

'P:\\Daten\\TU Darmstadt\\Capstone Project\\git\\capstone_project_git'

In [8]:
company_path = basedir + '/DNB Scraper/output final'

In [9]:
merged_company_files = [f for f in os.listdir(company_path) if os.path.isfile(os.path.join(company_path, f))]
merged_company_files

['company_attributes_Beratung_merged.json',
 'company_attributes_Dienstleistungen_merged.json',
 'company_attributes_IT_merged.json']

#### Show first file:

In [12]:
# company attributes: [Name, Mitarbeiter, Umsatz in Mio, PLZ, Stadt]
# new company attributes: [Alter Name, Neuer Name, Mitarbeiter, Umsatz in Mio, PLZ, Stadt]
# 'ss' zu 'ß' 

file_pathname = os.path.normpath(company_path + '/' + merged_company_files[0])

with open(file_pathname,'r') as file:
    company_attributes = json.load(file)

In [13]:
len(company_attributes)

4447

In [14]:
company_attributes

[['AKKA', 'AKKA DSO GmbH', '371', '36.04', '80807', ' MÜNchen'],
 ['wildstyle network GmbH',
  'Wildstyle Network GmbH',
  '28',
  '3.18',
  '01099',
  ' Dresden'],
 ['The Solving Company GmbH',
  'The Solving Company GmbH',
  '2',
  '0.31',
  '80538',
  ' MÜNchen'],
 ['Prosol IT GmbH', 'PROSOL IT GmbH', '3', '0.83', '22587', ' Hamburg'],
 ['dr. odin gmbh', 'Dr. Odin GmbH', '12', '1.7', '22393', ' Hamburg'],
 ['Cim-Base GmbH',
  'CIM-BASE GmbH Consulting Und Engineering',
  '12',
  '1.18',
  '71665',
  ' Vaihingen An Der Enz'],
 ['disquom funktechnik GmbH',
  'Disquom Funktechnik Gmbh',
  '13',
  '1.99',
  '53501',
  ' Grafschaft'],
 ['future impulse GmbH',
  'Future Impulse GmbH',
  '6',
  '0.35',
  '71034',
  ' BÖBlingen'],
 ['Glöckle direct GmbH',
  'GlÖCkle Direct GmbH',
  '156',
  '23.54',
  '70372',
  ' Stuttgart'],
 ['Stadtwerke Saarbrücken Consulting GmbH',
  'Stadtwerke SaarbrÜCken Consulting GmbH',
  '3',
  '0.7',
  '66117',
  ' SaarbrÜCken'],
 ['SIRE Life Sciences',
  'SIRE 

## 3.2 Scraping & save results

In [15]:
# 1150 comapnies ~ 10 min

### Problems:

- 'IndustrialPort Real Estate GmbH & Co. KG', 'IndustrialPort Real Estate GmbH & Co. KG', '2', '0.15', '65510', ' Idstein'
-> No court could be found.

In [16]:
for merged_file in merged_company_files:
    file_pathname = os.path.normpath(company_path + '/' + merged_file)

    with open(file_pathname, 'r') as file:
        company_attributes = json.load(file)
    
    print('File: ' + file_pathname)
    print('Number of companies: ' + str(len(company_attributes)))
    
    time.sleep(0.5)
    start = time.time()
    insolvenz_data_dict, errors, company_attributes_insolvency = helpers.scrape_court_and_insolvency(company_attributes, 
                                                                                                     insolvency_court, 
                                                                                                     time_out=0, 
                                                                                                     verbose=False)
    print('Scraping finished: ---' + str((time.time()-start)/60) + 'min ---')
    
    print('\nResults:')
    print('insolvenz_data_dict:' + str(len(insolvenz_data_dict)))
    print('errors:' + str(len(errors)))
    print('company_attributes_insolvency:' + str(len(company_attributes_insolvency)))
    
    if len(company_attributes) == len(company_attributes_insolvency):
        print('\nSave results to .json:')
        with open(basedir + '/Insolvency Scraper/data/' + 'insolvency_data_' + merged_file, 'w') as f:
            json.dump(insolvenz_data_dict, f)
        print('insolvenz_data_dict --saved--')

        with open(basedir + '/Insolvency Scraper/data/' + 'errors_' + merged_file, 'w') as f:
            json.dump(errors, f)
        print('errors --saved--')

        with open(basedir + '/Insolvency Scraper/data/' + 'new_' + merged_file, 'w') as f:
            json.dump(company_attributes_insolvency, f)
        print('company_attributes_insolvency --saved--')
        
    else: 
        print('\nNumber of companies from file: ' + str(len(company_attributes)) + 
              '\nNumber of companies with any results: ' + str(len(company_attributes_insolvency)))
        break
    print('')

File: P:\Daten\TU Darmstadt\Capstone Project\git\capstone_project_git\DNB Scraper\output final\company_attributes_Beratung_merged.json
Number of companies: 4447


[========================================================================] 100%


Scraping finished: ---38.897231761614485min ---

Results:
insolvenz_data_dict:84
errors:20
company_attributes_insolvency:4447

Save results to .json:
insolvenz_data_dict --saved--
errors --saved--
company_attributes_insolvency --saved--
File: P:\Daten\TU Darmstadt\Capstone Project\git\capstone_project_git\DNB Scraper\output final\company_attributes_Dienstleistungen_merged.json
Number of companies: 4435


[========================================================================] 100%


Scraping finished: ---37.7385634303093min ---

Results:
insolvenz_data_dict:136
errors:26
company_attributes_insolvency:4435

Save results to .json:
insolvenz_data_dict --saved--
errors --saved--
company_attributes_insolvency --saved--
File: P:\Daten\TU Darmstadt\Capstone Project\git\capstone_project_git\DNB Scraper\output final\company_attributes_IT_merged.json
Number of companies: 6897


Scraping finished: ---57.36037943760554min ---

Results:
insolvenz_data_dict:119
errors:24
company_attributes_insolvency:6897

Save results to .json:
insolvenz_data_dict --saved--
errors --saved--
company_attributes_insolvency --saved--


## 3.2 Read files:

In [10]:
print('Path: ' + basedir + '\n')
for merged_file in merged_company_files:
    
    max_len = len(merged_file)
    var_name = merged_file[19:max_len-5]

    with open(basedir + '/Insolvency Scraper/data/' + 'insolvency_data_' + merged_file, 'r') as f:
        insolvenz_data = json.load(f)
    vars()[var_name+'_insolvenz_data'] = insolvenz_data
    print(var_name+'insolvency_data_. ' + '(' + str(len(insolvenz_data)) + ')')
  
    with open(basedir + '/Insolvency Scraper/data/' + 'errors_' + merged_file, 'r') as f:
        error_data = json.load(f)
    vars()[var_name+'_errors'] = error_data
    print(var_name+'_errors. ' + '(' + str(len(error_data)) + ')')
        
    with open(basedir + '/Insolvency Scraper/data/' + 'new_' + merged_file, 'r') as f:
        company_attributes = json.load(f)
    vars()[var_name+'_company_attributes_insolvency'] = company_attributes
    print(var_name+'_company_attributes_insolvency. ' + '(' + str(len(company_attributes)) + ')\n')

Path: P:\Daten\TU Darmstadt\Capstone Project\git\capstone_project_git

Beratung_merged_insolvenz_data. (84)
Beratung_merged_errors. (20)
Beratung_merged_company_attributes_insolvency. (4447)

Dienstleistungen_merged_insolvenz_data. (136)
Dienstleistungen_merged_errors. (26)
Dienstleistungen_merged_company_attributes_insolvency. (4435)

IT_merged_insolvenz_data. (119)
IT_merged_errors. (24)
IT_merged_company_attributes_insolvency. (6897)



#### All insolvencies found in 'Dienstleistungen' as a list:

In [11]:
[*Dienstleistungen_merged_insolvenz_data]

['Instituts FÜR Analytik Und Schwachstellenforschung',
 'Schickor Personaldienstleistungs GmbH',
 'TelDaFax Network GmbH',
 'INOVA Competence GmbH',
 'Personaldienstleister',
 'Baumarkt Max Bahr Praktiker Einkaufs GmbH',
 'We Plan GmbH',
 'Thomas Binder',
 'VSD Sicherheitsdienste SÜDwest GmbH',
 'WKD Sicherheit GmbH',
 'DynaTrav Dynamic Travel Solutions GmbH',
 'P P Personal-Profi GmbH',
 'SMR Entsorgungs GmbH',
 'Signatur Planen Gestalten Einrichten Gesellschaft Mit BeschrÄNkter Haftung',
 'KnaufKassel GmbH',
 'JUMA Job GmbH',
 'I-Tegra Xperts GmbH',
 'Contact Personal Service GmbH',
 'Besser Personalmanagement GmbH',
 'Schirmherrschaft GmbH',
 'GETS-Logistik GmbH',
 'ITALWEG-Italienreisen GmbH',
 'Riede Stefan',
 "Theo'S Personal Vermietungs-Service GmbH",
 'Ernst & Young Cqc Corporate Quality Consulting GmbH',
 'HMW MÖBel GmbH',
 'Deutscher Spendenhilfsdienst - DSH GmbH',
 'Spa Group Europe GmbH',
 'Kristall Personalservice GmbH',
 'Alarm- Und GebÄUdetechnik SI-EX GmbH',
 'HÖRwick S

#### New company_attributes information for 'Dienstleistungen':

In [12]:
df = pd.DataFrame(Dienstleistungen_merged_company_attributes_insolvency, columns=['name (based on kununu)', 'name (based on dnb)', 'employee', 'sales Mio. €', 'postcode', 'city', 'insolvency', 'insolvency court', 'not relevant'])
df

,name (based on kununu),name (based on dnb),employee,sales Mio. €,postcode,city,insolvency,insolvency court,not relevant
0,hairdefinition,Hairdefinition GmbH,5,0.94,40789,Monheim Am Rhein,Düsseldorf,0,NaN
1,SVG Bundes-Zentralgenossenschaft Straßenverkeh...,SVG Bundes-Zentralgenossenschaft StraSSEnverke...,35,121.47,60487,Frankfurt Am Main,Frankfurt,0,NaN
2,AVANCE Personal Service GmbH,AVANCE Personal Service GmbH,600,20.83,01189,Dresden,Dresden,0,NaN
3,Bestattungshaus SARG-MÜLLER,Bestattungshaus SARG-MÜLLER Otto MÜLler E.K.,5,0.28,38106,Braunschweig,Braunschweig,0,NaN
4,DESK SICHERHEIT,Desk Sicherheit Unternehmergesellschaft (Haftu...,3,0.25,76137,Karlsruhe,Karlsruhe,0,NaN
...,...,...,...,...,...,...,...,...,...
4430,UMB GmbH,UMB GmbH,10,0.83,32760,Detmold,Detmold,0,NaN
4431,sandra rich,Sandra Rich GmbH,20,3.92,56424,Ebernhahn,Montabaur,0,NaN
4432,Insert Information Technologies GmbH,Insert Information Technologies GmbH,10,0.26,58456,Witten,Bochum,0,NaN
4433,RAK Ceramics GmbH,RAK Ceramics GmbH,39,32.06,60314,Frankfurt Am Main,Frankfurt,0,NaN


#### Company attributes filtered by insolvency:

In [15]:
df[df['insolvency court']==1]

,name (based on kununu),name (based on dnb),employee,sales Mio. €,postcode,city,insolvency,insolvency court,not relevant
22,Instituts für Analytik und Schwachstellenforsc...,Instituts FÜR Analytik Und Schwachstellenforsc...,2,0.14,68199,Mannheim,Mannheim,1,NaN
74,Schickor Personaldienstleistungs GmbH,Schickor Personaldienstleistungs GmbH,10,14.16,01097,Dresden,Dresden,1,NaN
114,TelDaFax Network GmbH,TelDaFax Network GmbH,5,0.74,53842,Troisdorf,Bonn,1,NaN
117,INOVA Competence GmbH,INOVA Competence GmbH,2,0.32,76870,Kandel,Landau in der Pfalz,1,NaN
154,Personaldienstleister,Personaldienstleister,2,0.14,85598,Baldham,München,1,NaN
...,...,...,...,...,...,...,...,...,...
4251,Wirtschaftsforum Künsting AG,Wirtschaftsforum KÜNsting AG,35,4.23,33100,Paderborn,Paderborn,1,NaN
4268,taurus media GmbH,Taurus Media GmbH,9,1.23,20457,Hamburg,Hamburg,1,NaN
4311,Powerwork KG,Powerwork KG,3,1.27,79761,Waldshut-Tiengen,Waldshut-Tiengen,1,NaN
4326,ELPRO Elektronik-Produkt Recycling GmbH,ELPRO Elektronik-Produkt Recycling GmbH,80,1.69,38116,Braunschweig,Braunschweig,1,NaN


errors = pd.DataFrame(Dienstleistungen_merged_errors, columns=['name (based on kununu)', 'name (based on dnb)', 'employee', 'sales Mio. €', 'postcode', 'city', 'error type', 'insolvency', 'not relevant'])
errors